In [1]:
from selenium import webdriver
import time
import json

#json으로 cookie를 cookie.json 저장하기
#한번은 실행하여 json 저장해야함
path = "C:\python_temp\chromedriver_win32_93.0\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

#     username=''
#     password= ''
username=''
password= ''
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(username)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(password)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(2)
try:
    driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
except:
    pass
time.sleep(2)
try:
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
except:
    pass
time.sleep(2)

#json으로 현 쿠키값 저장하기
with open('cookies.json','w') as cookie_json_w:
    json.dump(driver.get_cookies(),cookie_json_w)

In [2]:
# #사용함수
from selenium import webdriver
from bs4 import BeautifulSoup, NavigableString
import requests
import time
import pandas as pd
import datetime
import json
from selenium.webdriver.common.keys import Keys
import re
# import emoji

#미사용
# import pickle
# import os
# import urllib

# user2=[]
# img2=[]
# post2=[]
hasht2=[]
# hasht3=[]
# like2=[]
# day2=[]


path = "C:\python_temp\chromedriver_win32_93.0\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)
#전체화면
driver.maximize_window()
time.sleep(1)

#pickle로 cookie 불러오기
cookies=json.load(open('cookies.json','r'))
for cookie in cookies:
    driver.add_cookie(cookie)
time.sleep(1)
driver.refresh() #새로고침

time.sleep(2)
try:
    driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
    time.sleep(2)
except:
    pass


search=input('검색할 내용을 입력하세요 : ')
elements=driver.find_element_by_class_name('XTCLo')
elements.send_keys(search)
time.sleep(1)
elements.send_keys(Keys.ARROW_DOWN)
time.sleep(1)
elements.send_keys(Keys.ENTER)
time.sleep(5)

검색할 내용을 입력하세요 : 혜화역4번출구


In [5]:
##데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'location': [],
              'date': [],
              'like': [],
              'text': [],
              'hashtag': [],
              'img': []}


try: 
    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(0.05)
except:
    pass

#최근 게시물 첫번째로 선택
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div/div[1]/div[1]/a/div[1]/div[2]').click()
time.sleep(1)

#빨리하기위해서 10개로 제한
cnt=1
while cnt<400:
    time.sleep(0.05)
    page=driver.page_source
    soup=BeautifulSoup(page,'html.parser')
    time.sleep(0.05)
    
    try:
        soup.find('article','M9sTE').get_text()
    except AttributeError:
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/a[1]').click()
        time.sleep(30)
    else:
    
        #유저아이디
        try:
            user=soup.find('article','M9sTE').find('div','e1e1d').find('a').get_text()
        except AttributeError:
            user=''
        insta_dict['id'].append(user)
        time.sleep(0.05)

        #지역
        try:
            location=soup.find('article','M9sTE').find('div','JF9hh').find('a').get_text()
        except AttributeError:
            location=''
        insta_dict['location'].append(location)
        time.sleep(0.05)

        #이미지,영상

        if 'video' in soup.find('article','M9sTE'):
            try:
                img_src=soup.find('article','M9sTE').find('video')
            except AttributeError:
                img=''
            img=img_src['src']
            insta_dict['img'].append(img)
            time.sleep(0.05)

        else:
            try:
                img_src=soup.find('article','M9sTE').find('img')
            except AttributeError:
                img=''
            img=img_src['src']
            insta_dict['img'].append(img)
            time.sleep(0.05)

        #해시태그
        hasht2=[]
        try:
            hashts=soup.find('article','M9sTE').find_all('a','xil3i')
        except AttributeError:
            hasht=''
        for i in hashts:
            hasht=i.get_text()
            hasht2.append(hasht)
    #         ','.join(hasht2)
        insta_dict['hashtag'].append(hasht2)
        time.sleep(0.05)
    #     hasht2=[]

        #게시글
        try:
            posts=soup.find('article','M9sTE').find('div','X7jCj').find('div','C4VMK').find_all('span')[1]
        except AttributeError:
            post=''
        else:
            try:
                for tag in posts.find_all('a'):
                    tag.extract()
                post=posts.get_text()
            except AttributeError:
                post=''
            insta_dict['text'].append(post)
            time.sleep(0.05)

        #좋아요
        likes=soup.find('article','M9sTE')
        try:
            like=likes.find('a','zV_Nj').get_text()
        except AttributeError:
            like=''
        insta_dict['like'].append(like)
        time.sleep(0.05)

        #게시날짜
        day=soup.find('article','M9sTE').find('time','_1o9PC')['datetime']
        insta_dict['date'].append(day)
        time.sleep(0.05)

        print(cnt)
        cnt+=1

        driver.find_element_by_class_name('_65Bje').click()
        time.sleep(1)
    
    
with open('instagram_data.json','w') as f:
    json.dump(insta_dict,f,indent=4)

# save=pd.DataFrame(insta_dict)
# save.to_csv('insta{}.csv'.format(time.time()),encoding=('utf-8-sig'))


# save=pd.DataFrame({'아이디':user2,'이미지':img2,'게시글':post2,'해시태그':hasht3,'좋아요':like2,'게시시간':day2})
# save.to_excel('insta.xls',encoding=('UTF-8'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
